In [1]:
import pickle
import re
import pandas as pd

notes = pickle.load(open("pickle/notes.p", "rb" ))
links = pickle.load(open("pickle/links.p", "rb" ))

print(len(notes))
print(len(links))

87145
285557


In [8]:
def process_abc_note(note):
    lines = note.strip().splitlines()
    title = [x for x in lines if re.compile("T:").match(x)]
    title = title[0].split(":")[1].strip() if len(title)>0 else None

    time = [x for x in lines if re.compile("M:").match(x)]
    time = time[0].split(":")[1].strip() if len(time)>0 else None

    length = [x for x in lines if re.compile("L:").match(x)]
    length = length[0].split(":")[1].strip() if len(length)>0 else None

    tune_type = [x for x in lines if re.compile("R:").match(x)]
    tune_type = tune_type[0].split(":")[1].strip() if len(tune_type)>0 else None

    keyindex = [i for i, x in enumerate(lines) if re.compile("K:").match(x)]
    key = lines[keyindex[0]].split(":")[1].strip() if len(keyindex)>0 else None
    keyindex = 0 if len(keyindex)==0 else keyindex[0]

    tune = [x for x in lines[keyindex:] if re.compile(".*\\|").match(x)]
    tune = "".join(tune) if len(tune)>0 else None
    
    barsearch = re.compile(":?\|+:?\]?")
    bars = None
    if tune is not None:
        bars = [bar for bar in barsearch.split(tune) if bar!=""]
        barlines = barsearch.findall(tune)
        bars = [b+l for b, l in zip(bars,barlines)]

    features = {
        "title": title,
        "time": time,
        "length": length,
        "tune_type": tune_type,
        "key": key,
        "tune": tune,
        "bars": bars,
        "opening": None,
        "middle": None,
        "ending": None,
        "full_text": note
    }
    
    if bars is not None:
        features["opening"] = bars[:4]
        features["middle"] = bars[-4:4]
        features["ending"] = bars[-4:]
    
    features["complete"] = None not in features.values()
    return(features)

In [11]:
features = [process_abc_note(note) for note in notes]
pickle.dump(features, open("pickle/features.p", "wb"))
len(features)

87145

In [12]:
complete_features = [x for x in features if x['complete']]

pickle.dump(complete_features, open("pickle/complete_features.p", "wb"))
len(complete_features)

45150